In [338]:
import requests
import json
import geopandas as gpd
import pandas as pd
import folium
import branca

## Pinging the OTP Server

Once I built and started the OpenTripPlanner server, I wrote a function to ping the server for the isochrone data. When building the url, I also need to add all the parameters.

In [339]:
def analyst():

    coords  = '43.6629,-79.3957'
    date = '06-08-2020'
    bins = [1800,3600,5400,7200,9000,10800]
    cutoff = f'&cutoffSec={bins[0]}&cutoffSec={bins[1]}&cutoffSec={bins[2]}&cutoffSec={bins[3]}&cutoffSec={bins[4]}&cutoffSec={bins[5]}'
    tm = '08:%2030am'
    walkReluctance = 2
    walk_dist = 1600
    transfer_time=180
    mode = 'WALK,TRANSIT'
    params = f'?toPlace={coords}&fromPlace={coords}&arriveBy=TRUE&mode={mode}&date={date}&time={tm}&maxWalkDistance={walk_dist}&walkReluctance={walkReluctance}&minTransferTime={transfer_time}'
    url = f'http://localhost:8080/otp/routers/current/isochrone'+params+cutoff
    response = requests.get(url)

    return(response.json())

The data gets returned as a json file, which I'll save to my computer.

In [340]:
data = analyst()
with open('data.geojson', 'w') as outfile:
    json.dump(data, outfile)

## Cleaning the Data

I load the file back into memory using `geopandas`.

In [341]:
isochrone = gpd.read_file('data.geojson')

The polygons outputted from OpenTripPlanner overlapped the other polygons, meaning the 10800 second polygon included the entirety of the 1800 second polygon. I'm clipping the polygons to make sure none of the polygons overlap with each other, which should make customizing my map easier.

In [372]:
df30 = isochrone[isochrone['time']==1800]
df60 = gpd.overlay(isochrone[isochrone['time']==3600],isochrone[isochrone['time']==1800], how='difference')
df90 = gpd.overlay(isochrone[isochrone['time']==5400],isochrone[isochrone['time']==3600], how='difference')
df120 = gpd.overlay(isochrone[isochrone['time']==7200],isochrone[isochrone['time']==5400], how='difference')
df150 = gpd.overlay(isochrone[isochrone['time']==9000],isochrone[isochrone['time']==7200], how='difference')
df180 = gpd.overlay(isochrone[isochrone['time']==10800],isochrone[isochrone['time']==9000], how='difference')

## Mapping

Next I'll use `folium` to map the daata. I'm loading each isochrone polygon as a separate element, and manually adjusting the colour of each polygon. 

`folium` does offer a function to create a chloropleth map, however I couldn't get the legend to look the way I wanted.

In [375]:
m = folium.Map(location=[43.6629,-79.3957], zoom_start=9)

folium.GeoJson(
    df30,
    name='30min',
    style_function = lambda x:{
        'fillOpacity': 0.6,
        'weight': 0.2,
        'fillColor': '#eff3ff',
        'color': 'black'
    }
).add_to(m)
folium.GeoJson(
    df60,
    name='60min',
    style_function = lambda x:{
        'fillOpacity': 0.6,
        'weight': 0.2,
        'fillColor': '#c6dbef',
        'color': 'black'
    }
).add_to(m)
folium.GeoJson(
    df90,
    name='90min',
    style_function = lambda x:{
        'fillOpacity': 0.6,
        'weight': 0.2,
        'fillColor': '#9ecae1',
        'color': 'black'
    }
).add_to(m)
folium.GeoJson(
    df120,
    name='120min',
    style_function = lambda x:{
        'fillOpacity': 0.6,
        'weight': 0.2,
        'fillColor': '#6baed6',
        'color': 'black'
    }
).add_to(m)
folium.GeoJson(
    df150,
    name='150min',
    style_function = lambda x:{
        'fillOpacity': 0.6,
        'weight': 0.2,
        'fillColor': '#3182bd',
        'color': 'black'
    }
).add_to(m)
folium.GeoJson(
    df180,
    name='180min',
    style_function = lambda x:{
        'fillOpacity': 0.6,
        'weight': 0.2,
        'fillColor': '#08519c',
        'color': 'black'
    }
).add_to(m)

This giant chunk of html code is purely to format the legend. IIRC, you can use html to customize the map. I borrowed the code to make the legend from [here](https://tilemill-project.github.io/tilemill/docs/guides/advanced-legends/) and [here](https://nbviewer.jupyter.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd)

In [374]:
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Transit Travel Time From UofT (Minutes)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#eff3ff;opacity:0.7;'></span>0 to 30 Minutes</li>
    <li><span style='background:#c6dbef;opacity:0.7;'></span>30 to 60 Minutes</li>
    <li><span style='background:#9ecae1;opacity:0.7;'></span>60 to 90 Minutes</li>
    <li><span style='background:#6baed6;opacity:0.7;'></span>90 to 120 Minutes</li>
    <li><span style='background:#3182bd;opacity:0.7;'></span>120 to 150 Minutes</li>
    <li><span style='background:#08519c;opacity:0.7;'></span>150 to 180 Minutes</li>

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

m.get_root().add_child(macro)